# 東京家賃予測 - Deep Learning モデル訓練

このノートブックでは、東京の家賃データを使用してディープラーニングモデルを訓練します。

## 訓練の流れ
1. データ読み込みと前処理
2. モデルの作成（基本モデルとAttentionモデル）
3. モデルの訓練
4. モデルの評価と比較
5. 基本的な可視化

## 1. ライブラリのインポート

In [ ]:
# 日本語フォント設定
from rent_utils import setup_japanese_font
setup_japanese_font()

import matplotlib.pyplot as plt

print("✅ 日本語フォント設定完了")

import pandas as pd
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# カスタムモジュール
from rent_dl_data import RentDataPreprocessor, TokyoRentDataset
from rent_dl_models import RentPredictionNet, RentPredictionNetWithAttention
from rent_dl_train import train_model, evaluate_model
from rent_config import DEVICE

print(f"Device: {DEVICE}")

## 2. データ読み込みと前処理

In [ ]:
# データ読み込み
df = pd.read_csv('tokyo_rent_data_v2.csv')
print(f"データ数: {len(df)}")
print(f"\nデータの先頭:")
df.head()

In [ ]:
# データ前処理
preprocessor = RentDataPreprocessor()
df_processed = preprocessor.fit_transform(df)

print("前処理完了!")
print(f"区の数: {preprocessor.num_wards}")
print(f"建物構造の種類: {len(preprocessor.label_encoders['建物構造'].classes_)}")
print(f"建物タイプの種類: {len(preprocessor.label_encoders['建物タイプ'].classes_)}")

In [ ]:
# 特徴量とターゲットの分離
feature_cols = ['区_encoded', '建物構造_encoded', '建物タイプ_encoded',
                '部屋サイズ_m2', '駅距離_分', '築年数_年', '区_avg_price']

X = df_processed[feature_cols].values
y = df['家賃_円'].values / 10000  # 1万円単位にスケール

# データ分割 (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"訓練データ: {len(X_train)}")
print(f"検証データ: {len(X_val)}")
print(f"テストデータ: {len(X_test)}")

In [ ]:
# DataLoaderの作成
train_dataset = TokyoRentDataset(X_train, y_train)
val_dataset = TokyoRentDataset(X_val, y_val)
test_dataset = TokyoRentDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("DataLoader作成完了!")

## 3. モデルの作成

In [ ]:
# モデルパラメータ
num_wards = len(preprocessor.label_encoders['区'].classes_)
num_structures = len(preprocessor.label_encoders['建物構造'].classes_)
num_types = len(preprocessor.label_encoders['建物タイプ'].classes_)

print(f"num_wards: {num_wards}")
print(f"num_structures: {num_structures}")
print(f"num_types: {num_types}")

In [ ]:
# 基本モデルの作成
model_basic = RentPredictionNet(num_wards, num_structures, num_types).to(DEVICE)

total_params = sum(p.numel() for p in model_basic.parameters())
print(f"基本モデル総パラメータ数: {total_params:,}")

In [ ]:
# Attentionモデルの作成
model_attention = RentPredictionNetWithAttention(num_wards, num_structures, num_types).to(DEVICE)

total_params = sum(p.numel() for p in model_attention.parameters())
print(f"Attentionモデル総パラメータ数: {total_params:,}")

## 4. モデルの訓練

In [ ]:
# 基本モデルの訓練
print("基本モデルを訓練中...")
train_losses_basic, val_losses_basic = train_model(
    model_basic, train_loader, val_loader, DEVICE,
    num_epochs=50, learning_rate=0.001,
    model_save_path='best_rent_model_basic.pth'
)

print("\n基本モデル訓練完了!")

In [ ]:
# Attentionモデルの訓練
print("Attentionモデルを訓練中...")
train_losses_att, val_losses_att = train_model(
    model_attention, train_loader, val_loader, DEVICE,
    num_epochs=50, learning_rate=0.001,
    model_save_path='best_rent_model_attention.pth'
)

print("\nAttentionモデル訓練完了!")

## 5. モデルの評価

In [ ]:
# ベストモデルをロード
model_basic.load_state_dict(torch.load('best_rent_model_basic.pth', map_location=DEVICE, weights_only=True))
model_attention.load_state_dict(torch.load('best_rent_model_attention.pth', map_location=DEVICE, weights_only=True))

print("ベストモデルをロードしました")

In [ ]:
# 基本モデルの評価
predictions_basic, actuals_basic, mae_basic, rmse_basic, r2_basic = evaluate_model(
    model_basic, test_loader, DEVICE, "基本モデル"
)

In [ ]:
# Attentionモデルの評価
predictions_att, actuals_att, mae_att, rmse_att, r2_att = evaluate_model(
    model_attention, test_loader, DEVICE, "Attentionモデル"
)

In [ ]:
# モデル比較
print("\n" + "=" * 80)
print("モデル性能比較")
print("=" * 80)
print(f"{'指標':<15} {'基本モデル':<20} {'Attentionモデル':<20} {'改善率':<15}")
print("-" * 80)
print(f"{'MAE (¥)':<15} {mae_basic:>15,.0f}    {mae_att:>15,.0f}    {(mae_basic-mae_att)/mae_basic*100:>10.2f}%")
print(f"{'RMSE (¥)':<15} {rmse_basic:>15,.0f}    {rmse_att:>15,.0f}    {(rmse_basic-rmse_att)/rmse_basic*100:>10.2f}%")
print(f"{'R² Score':<15} {r2_basic:>15.4f}    {r2_att:>15.4f}    {(r2_att-r2_basic)/r2_basic*100:>10.2f}%")

## 6. 基本的な可視化

In [ ]:
# 日本語フォント再設定
from rent_utils import setup_japanese_font
setup_japanese_font()

# 訓練曲線と予測結果の可視化
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. 訓練曲線（基本モデル）
axes[0, 0].plot(train_losses_basic, label='Train Loss', alpha=0.7)
axes[0, 0].plot(val_losses_basic, label='Validation Loss', alpha=0.7)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training History - Basic Model')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 訓練曲線（Attentionモデル）
axes[0, 1].plot(train_losses_att, label='Train Loss', alpha=0.7)
axes[0, 1].plot(val_losses_att, label='Validation Loss', alpha=0.7)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Training History - Attention Model')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 予測 vs 実際（Attentionモデル）
axes[1, 0].scatter(actuals_att, predictions_att, alpha=0.5)
axes[1, 0].plot([actuals_att.min(), actuals_att.max()],
                [actuals_att.min(), actuals_att.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Rent (¥)')
axes[1, 0].set_ylabel('Predicted Rent (¥)')
axes[1, 0].set_title(f'Predictions vs Actual (R² = {r2_att:.4f})')
axes[1, 0].grid(True, alpha=0.3)

# 4. 残差プロット（Attentionモデル）
residuals = actuals_att - predictions_att
axes[1, 1].scatter(predictions_att, residuals, alpha=0.5)
axes[1, 1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 1].set_xlabel('Predicted Rent (¥)')
axes[1, 1].set_ylabel('Residuals (¥)')
axes[1, 1].set_title('Residual Plot')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("可視化保存: training_results.png")

In [ ]:
# 日本語フォント再設定
from rent_utils import setup_japanese_font
setup_japanese_font()

# 区の埋め込み可視化（Attentionモデル）
ward_embeddings = model_attention.ward_embedding.weight.detach().cpu().numpy()
ward_names = preprocessor.label_encoders['区'].classes_

# PCA 2D投影
pca = PCA(n_components=2)
ward_embeddings_2d = pca.fit_transform(ward_embeddings)

# 区別平均価格で色付け
ward_avg_prices = df.groupby('区')['家賃_円'].mean()
colors = [ward_avg_prices[ward] for ward in ward_names]

plt.figure(figsize=(12, 8))
scatter = plt.scatter(ward_embeddings_2d[:, 0], ward_embeddings_2d[:, 1],
                      c=colors, cmap='RdYlBu_r', s=200, alpha=0.7)

# いくつかの区名を表示
for i, ward in enumerate(ward_names):
    if ward in ['港区', '千代田区', '渋谷区', '新宿区', '足立区', '葛飾区', '江戸川区']:
        plt.annotate(ward, (ward_embeddings_2d[i, 0], ward_embeddings_2d[i, 1]),
                     fontsize=10, ha='center')

plt.xlabel('Embedding Dimension 1')
plt.ylabel('Embedding Dimension 2')
plt.title('Ward Embeddings Visualization (PCA 2D)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Average Rent (¥)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('ward_embeddings_2d.png', dpi=300, bbox_inches='tight')
plt.show()

print("可視化保存: ward_embeddings_2d.png")

## 7. モデルとpreprocessorの保存

In [ ]:
# 基本モデルの保存
torch.save({
    'model_state_dict': model_basic.state_dict(),
    'preprocessor': preprocessor,
    'model_config': {
        'num_wards': num_wards,
        'num_structures': num_structures,
        'num_types': num_types
    }
}, 'rent_prediction_model_basic.pth')

# Attentionモデルの保存
torch.save({
    'model_state_dict': model_attention.state_dict(),
    'preprocessor': preprocessor,
    'model_config': {
        'num_wards': num_wards,
        'num_structures': num_structures,
        'num_types': num_types
    }
}, 'rent_prediction_model_attention.pth')

print("モデル保存完了!")
print("  - rent_prediction_model_basic.pth")
print("  - rent_prediction_model_attention.pth")

## 完了!

訓練が完了しました。次のノートブックで予測や可視化を行えます:
- `rent_dl_predict.ipynb` - 予測と対話型UI
- `rent_dl_visualize.ipynb` - 高度な可視化と分析